In [ ]:
 # imports
# !pip install spacy==2.3.5
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# import spacy


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
import spacy

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = pd.read_csv('reddit_wsb.csv')
data.head()

# nyseStocks = pd.read_csv('nyse.csv')
# nasdaqStocks = pd.read_csv('nasdaq.csv')

data.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1611862661,NaN,1/28/2021 21:37
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1611862330,NaN,1/28/2021 21:32
2,Exit the system GME,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1611862235,The CEO of NASDAQ pushed to halt trading “to g...,1/28/2021 21:30
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1611862137,NaN,1/28/2021 21:28
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1611862016,NaN,1/28/2021 21:26


In [ ]:
data['length'] = [len(i.split()) for i in data['title']]
data['length'].min()

1

In [ ]:
#Contains a list of every ticker tape used on the NYSE and NASDAQ
# stocks = set(nyseStocks['Symbol'])
# stocks.update(set(nasdaqStocks['Symbol']))


In [ ]:
data = data[data['length'] > 3].copy()

In [ ]:
data['length'].min()

4

In [ ]:
def isUppercase(word):
  return word.isupper()

In [ ]:
#RUN ONLY ONCE.  DATA IS IN title_data.csv 

title_data = data[['title','timestamp']].copy()
title_data = title_data.dropna()
nlp = spacy.load("en_core_web_sm")

# tickerList = []

# for index, row in title_data.iterrows():
#   words = nlp(row['title'])
#   token_list = [token.orth_ for token in words]
#   wordset = set(filter(isUppercase, token_list))
#   tempTickers = (wordset.intersection(stocks))
#   #print(tempTickers)
#   tickerList.append(tempTickers)
#   #title_data[index, 'stocks'] = tempTickers
# print(tickerList)
# title_data['tickers'] = pd.Series(tickerList).values

title_data.title = title_data.title.str.lower()
title_data.title = title_data.title.apply(lambda x:re.sub('@[^\s]+','',x))
title_data.title = title_data.title.apply(lambda x:re.sub(r"http\S+", "", x))
title_data.timestamp = pd.to_datetime(title_data.timestamp).dt.date

In [ ]:
#Download the code for the first time after running the prev block

# title_data.head()

# from google.colab import files

# title_data.to_csv('title_data.csv')

# files.download("title_data.csv")

In [ ]:
# title_data = pd.read_csv('title_data.csv')

In [ ]:
import nltk
nltk.download('vader_lexicon')

sid = SIA()

title_data['sentiments'] = title_data['title'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
title_data['Positive Sentiment'] = title_data['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
title_data['Neutral Sentiment'] = title_data['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
title_data['Negative Sentiment'] = title_data['sentiments'].apply(lambda x: x['neg']+1*(10**-6))
title_data.drop(columns=['sentiments'],inplace=True)
title_data.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,title,timestamp,Positive Sentiment,Neutral Sentiment,Negative Sentiment
0,"it's not about the money, it's about sending a...",2021-01-28,0.000001,1.000001,0.000001
1,math professor scott steiner says the numbers ...,2021-01-28,0.000001,0.728001,0.272001
2,exit the system gme,2021-01-28,0.000001,1.000001,0.000001
3,new sec filing for gme! can someone less retar...,2021-01-28,0.138001,0.658001,0.204001
4,"not to distract from gme, just thought our amc...",2021-01-28,0.119001,0.881001,0.000001


In [ ]:
title_data['class_name'] = title_data[['Positive Sentiment', 'Neutral Sentiment', 'Negative Sentiment']].idxmax(axis=1)

In [ ]:
data = title_data[title_data['class_name'] != 'Neutral Sentiment']
data.reset_index(drop=True, inplace=True)
data.head(5)

,title,timestamp,Positive Sentiment,Neutral Sentiment,Negative Sentiment,class_name
0,i love you retards!!!!,2021-01-28,0.677001,0.323001,0.000001,Positive Sentiment
1,the wallstreetbets war museum,2021-01-28,0.000001,0.435001,0.565001,Negative Sentiment
2,this took me longer than i'd like to admit. en...,2021-01-28,0.517001,0.483001,0.000001,Positive Sentiment
3,most recent short interest,2021-01-28,0.521001,0.479001,0.000001,Positive Sentiment
4,i love the movies: part 1,2021-01-28,0.583001,0.417001,0.000001,Positive Sentiment


In [ ]:
data['class_name'][data['class_name'] == 'Negative Sentiment'] = 0
data['class_name'][data['class_name'] == 'Positive Sentiment'] = 1
data.rename(columns={'class_name':'class'}, inplace=True)

In [ ]:
data = data[['title','class']].copy()
data.head()

,title,class
0,i love you retards!!!!,1
1,the wallstreetbets war museum,0
2,this took me longer than i'd like to admit. en...,1
3,most recent short interest,1
4,i love the movies: part 1,1


In [ ]:
data['length'] = [len(i.split()) for i in data['title']]

In [ ]:
data['length'].min()

3

In [ ]:
!pip install transformers==3.0

     |████████████████████████████████| 757kB 18.8MB/s 
     |████████████████████████████████| 901kB 52.2MB/s 
     |████████████████████████████████| 1.2MB 54.7MB/s 
     |████████████████████████████████| 3.0MB 50.8MB/s 


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [ ]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(list(data['title']), list(data['class']), train_size=0.8, random_state=1, stratify=data['class'])

In [ ]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    sentences_train,
    max_length = 8,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    sentences_test,
    max_length = 8,
    pad_to_max_length=True,
    truncation=True
)

In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(y_train) 

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(y_test) 

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)

In [ ]:
for param in bert.parameters():
    param.requires_grad = True

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      super(BERT_Arch, self).__init__()
      self.bert = bert       
      self.dropout = nn.Dropout(0.1)      
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)      
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)      
      x = self.softmax(x)
      return x

In [ ]:
model = BERT_Arch(bert)
model = model.to(device)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-5)         

In [ ]:
cross_entropy  = nn.NLLLoss() 
epochs = 5

In [ ]:
def train():
  
  model.train()
  total_loss, total_accuracy = 0, 0  
  total_preds=[]
  
  for step,batch in enumerate(train_dataloader):    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    model.zero_grad()        
    preds = model(sent_id, mask)
    loss = cross_entropy(preds, labels)
    total_loss = total_loss + loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    preds=preds.detach().cpu().numpy()
    total_preds.append(preds)
  avg_loss = total_loss / len(train_dataloader)
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds


In [ ]:
def evaluate():

  print("\nEvaluating...")
  
  model.eval()
  total_loss, total_accuracy = 0, 0  
  total_preds = []
  
  for step,batch in enumerate(test_dataloader):    
    if step % 50 == 0 and not step == 0:      
      elapsed = format_time(time.time() - t0)            
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    batch = [t.to(device) for t in batch]
    sent_id, mask, labels = batch
    with torch.no_grad():      
      preds = model(sent_id, mask)
      loss = cross_entropy(preds,labels)
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
      total_preds.append(preds)
  avg_loss = total_loss / len(test_dataloader) 
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds


In [ ]:
best_valid_loss = float('inf')
train_losses=[]
valid_losses=[]
for epoch in range(epochs):
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))    
    train_loss, _ = train()
    valid_loss, _ = evaluate()    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')




 Epoch 1 / 5

Evaluating...

Training Loss: 0.679
Validation Loss: 0.664

 Epoch 2 / 5

Evaluating...

Training Loss: 0.608
Validation Loss: 0.508

 Epoch 3 / 5

Evaluating...

Training Loss: 0.426
Validation Loss: 0.427

 Epoch 4 / 5

Evaluating...

Training Loss: 0.297
Validation Loss: 0.408

 Epoch 5 / 5

Evaluating...

Training Loss: 0.223
Validation Loss: 0.466


In [ ]:
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
y_test = np.array(y_test)

In [ ]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       153
           1       0.85      0.88      0.86       218

    accuracy                           0.83       371
   macro avg       0.83      0.82      0.83       371
weighted avg       0.83      0.83      0.83       371



In [ ]:
df = pd.DataFrame({'text': list(sentences_test), 'sentiment': preds})

In [ ]:
df

,text,sentiment
0,i love all of you.,1
1,amc yolo update.. ape love movies 🦍🦍🦍,1
2,"my own ""if gme hits.."" promise that i hope i f...",1
3,i like the stock and free tendies,1
4,gme the high risk high reward play,0
...,...,...
366,a bit delayed here in oz but i'm fucking in yo...,0
367,32 gme shares at $250 a share,1
368,nok isn't banned 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀,1
369,dont let the vultures win on amc.,0
